# Transfer Learning YOLOv2 on LISA Dataset
---------------------------------------------------------------------


* What is Transfer learning
* Relationship between COCO and LISA

In [1]:
import sys
print(sys.version) # Check Python Version
import os
from keras.optimizers import Adam
from cfg import *

3.6.1 |Continuum Analytics, Inc.| (default, Mar 22 2017, 19:54:23) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


Using TensorFlow backend.


### Prepare LISA Dataset

In [2]:
import numpy as np
from utils.parse_input import load_data    # Data handler for LISA dataset

x_train, y_train = load_data('training.txt')
labels           = np.unique(y_train[:,1])
num_classes      = len(labels)            # Count number of classes in the dataset

print("Train: {} samples\nNumber of classes: {}".format(len(x_train),num_classes))
print("\nLabel Sample: \n{}".format(y_train[0]))


Number of ground truth boxes: 3672 boxes
Train: 3672 samples
Number of classes: 31

Label Sample: 
[1093.5 408.0 45.0 48.0 'doNotEnter']


### Load Pretrained YOLOv2 on COCO Dataset

In [3]:
from model.yolov2 import YOLOv2, darknet19
import keras.backend as K
K.clear_session() # Avoid duplicate model 

# Original YOLOv2 - Trained on COCO Dataset

darknet19 = darknet19(freeze_layers=False)
yolov2    = YOLOv2(feature_extractor=darknet19, num_anchors=5, num_classes=80)
# print(yolov2.model.summary())

## Transfer Learning - Cut last layer and create a new Conv2D for Traffic Sign Detection

In [4]:
from keras.layers import Conv2D
from keras.models import Model
from keras.regularizers import l2
# Transfer Learning -RISSNet based on YOLOv2 by 
# Cut last layer
topless_model = Model(yolov2.model.input, yolov2.model.layers[-2].output)

# Create new Conv2D layer for Traffic Sign Detection
traffic_sign_layer = Conv2D(filters=(N_ANCHORS* (N_CLASSES + 5)),
                            kernel_size=(1, 1), kernel_regularizer=l2(5e-4), name='yolov2')(topless_model.output)

# # Freeze topless layers
# for layer in topless_model.layers[:-16]:
#     layer.trainable = False
    
# Build a new Model for training
model = Model(yolov2.model.input, traffic_sign_layer)
model.load_weights('../yolo-coco.h5', by_name=True)

# model.summary()
# for l in model.layers:
#     print(l.trainable)

## Loss Function

In [5]:
import keras.backend as K
import tensorflow as tf

def custom_loss(y_true, y_pred):
    
    SCALE_NOOB, SCALE_CONF, SCALE_COOR, SCALE_PROB = 0.5, 5.0, 5.0, 1.0
    pred_shape = K.shape(y_pred)[1:3]
    gt_shape   = K.shape(y_true)                 # shape of ground truth value
    GRID_H = tf.cast(pred_shape[0], tf.int32)  # shape of output feature map
    GRID_W = tf.cast(pred_shape[1], tf.int32)
    
    output_size = tf.cast(tf.reshape([GRID_W, GRID_H], [1,1,1,1,2]), tf.float32)
    y_pred = tf.reshape(y_pred,[-1, pred_shape[0], pred_shape[1], N_ANCHORS, N_CLASSES +5])
    y_true = tf.reshape(y_true,[-1, gt_shape[1], gt_shape[2], N_ANCHORS, N_CLASSES + 5])

    c_xy = _create_offset_map(K.shape(y_pred))
    
    pred_box_xy = (tf.sigmoid(y_pred[:,:,:,:,:2]) + c_xy)  / output_size
    pred_box_wh = tf.exp(y_pred[:,:,:,:,2:4]) * np.reshape(ANCHORS, [1,1,1,N_ANCHORS,2])
    pred_box_wh = tf.sqrt(pred_box_wh / output_size)
    
    # adjust confidence
    pred_box_conf = tf.expand_dims(tf.sigmoid(y_pred[:, :, :, :, 4]), -1) 
    # adjust probability
    pred_box_prob = tf.nn.softmax(y_pred[:, :, :, :, 5:])
    
    y_pred = tf.concat([pred_box_xy, pred_box_wh, pred_box_conf, pred_box_prob], 4)
    
    ### Adjust ground truth
    center_xy = y_true[:,:,:,:,0:2]
    true_box_xy = center_xy 
    true_box_wh = tf.sqrt(y_true[:,:,:,:,2:4])
    
    # adjust confidence
    pred_tem_wh = tf.pow(pred_box_wh, 2) *  output_size
    pred_box_area = pred_tem_wh[:,:,:,:,0] * pred_tem_wh[:,:,:,:,1]
    pred_box_ul = pred_box_xy - 0.5 * pred_tem_wh
    pred_box_bd = pred_box_xy + 0.5 * pred_tem_wh
    
    true_tem_wh = tf.pow(true_box_wh, 2) *  output_size
    true_box_area = true_tem_wh[:,:,:,:,0] * true_tem_wh[:,:,:,:,1]
    true_box_ul = true_box_xy - 0.5 * true_tem_wh
    true_box_bd = true_box_xy + 0.5 * true_tem_wh
    
    intersect_ul = tf.maximum(pred_box_ul, true_box_ul) 
    intersect_br = tf.minimum(pred_box_bd, true_box_bd)
    intersect_wh = intersect_br - intersect_ul
    intersect_wh = tf.maximum(intersect_wh, 0.0)
    intersect_area = intersect_wh[:,:,:,:,0] * intersect_wh[:,:,:,:,1]
    
    iou = tf.truediv(intersect_area, true_box_area + pred_box_area - intersect_area)
    best_box = tf.equal(iou, tf.reduce_max(iou, [3], True)) 
    best_box = tf.to_float(best_box)
    true_box_conf = tf.expand_dims(best_box * y_true[:,:,:,:,4], -1)
    
    # adjust confidence
    true_box_prob = y_true[:,:,:,:,5:]
    
    y_true = tf.concat([true_box_xy, true_box_wh, true_box_conf, true_box_prob], 4)
    
    ### Compute the weights
    weight_coor = tf.concat(4 * [true_box_conf], 4)
    weight_coor = SCALE_COOR * weight_coor
    weight_conf = SCALE_NOOB * (1. - true_box_conf) + SCALE_CONF * true_box_conf
    weight_prob = tf.concat(N_CLASSES * [true_box_conf], 4) 
    weight_prob = SCALE_PROB * weight_prob 
    
    weight = tf.concat([weight_coor, weight_conf, weight_prob], 4)
    
    ### Finalize the loss
    loss = tf.pow(y_pred - y_true, 2)
    loss = loss * weight
    loss = tf.reshape(loss, [-1, tf.cast(GRID_W*GRID_H, tf.int32)*N_ANCHORS*(4 + 1 + N_CLASSES)])
    loss = tf.reduce_sum(loss, 1)
    loss = .5 * tf.reduce_mean(loss)
    
    return loss

def _create_offset_map(output_shape):
    """
    In Yolo9000 paper, grid map
    """
    GRID_H = tf.cast(output_shape[1], tf.int32)  # shape of output feature map
    GRID_W = tf.cast(output_shape[2], tf.int32)
    
    cx = tf.cast((K.arange(0, stop=GRID_W)), dtype=tf.float32)
    cx = K.tile(cx, [GRID_H])
    cx = K.reshape(cx, [-1, GRID_H, GRID_W, 1])

    cy = K.cast((K.arange(0, stop=GRID_H)), dtype=tf.float32)
    cy = K.reshape(cy, [-1, 1])
    cy = K.tile(cy, [1, GRID_W])  
    cy = K.reshape(cy, [-1])    
    cy = K.reshape(cy, [-1, GRID_H, GRID_W, 1])

    c_xy = tf.stack([cx, cy], -1)
    c_xy = K.cast(c_xy, tf.float32)

    return c_xy

In [6]:
import os
import keras
# from sklearn.model_selection import train_test_split

from utils.multi_gpu import make_parallel, get_gpus
from utils.data_generator import flow_from_list

# # HYPER-PARAMETERS
BATCH_SIZE = 8
EPOCHS     = 20
LEARN_RATE = 0.0001 # this model has been pre-trained, LOWER LR is needed

# Data Generator
train_data_gen = flow_from_list(x_train, y_train, batch_size=BATCH_SIZE, augment_data=True)

backup_path = '/home/ubuntu/dataset/backup/'
# For Debugging purpose
tf_board   = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)
early_stop = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.000001, patience=3, mode='min', verbose=1)
save_model = keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch,logs: model.save_weights(backup_path + 'yolov2-epoch%s-loss%s.weights'%(epoch, str(logs.get('loss')))))

In [7]:
from keras.optimizers import Adam, SGD

# Load pretrain weights
model.load_weights("yolov2.weights")

# Set up Data parallelism
n_gpus = get_gpus()
if n_gpus > 1:
    BATCH_SIZE = n_gpus * BATCH_SIZE
    model_par = make_parallel(model, n_gpus)
else:
    model_par = model

# Set up optimizer
# sgd = SGD(LEARN_RATE, decay=0.0005, momentum=0.9)
adam = Adam(LEARN_RATE)
# Compiling model
model_par.compile(optimizer=adam,loss=custom_loss)

hist =  model_par.fit_generator(generator     = train_data_gen, 
                            steps_per_epoch   = 200, 
                            epochs            = 2, 
                            callbacks         = [tf_board, early_stop, save_model],
                            workers=1, verbose=1, max_q_size=3)

model.save_weights('yolov2.weights')

There is(are) 1 GPU(s) on device.
Epoch 1/2


ResourceExhaustedError: OOM when allocating tensor with shape[8,480,640,64]
	 [[Node: batch_normalization_2/moments/sufficient_statistics/Sub = Sub[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](conv2d_2/convolution, batch_normalization_2/moments/StopGradient)]]
	 [[Node: batch_normalization_7/moments/sufficient_statistics/Gather/_1107 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_3812_batch_normalization_7/moments/sufficient_statistics/Gather", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'batch_normalization_2/moments/sufficient_statistics/Sub', defined at:
  File "/home/ubuntu/anaconda2/envs/yad2k/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda2/envs/yad2k/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_glob

## Multi-GPUs Training - Data Parallelism Approach

* Each GPU will have a copy of the model
* During training time, mean of all gradidents from each GPU will be calculated to update the model
<img style="width:40%" src="https://www.tensorflow.org/images/Parallelism.png">

In [ ]:
model.load_weights('yolov2.weights')

### Visualize training process using Tensorboard
Open `http://<public-dns>:6006`

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# samples, labels = train_data_gen.next()
# i = 2
# img    = samples[i]
# labels = labels.reshape([-1, 30, 40, 5, 36])
# box    = labels[i][8][30][0][:4]*np.array([1280, 960, 1280, 960])
# print(box)

# dot    = cv2.circle(img, (int(box[0]), int(box[1])), radius=5, color=[0, 1, 0], thickness=5)

# plt.figure(figsize=(10, 10))
# plt.imshow(dot)

In [ ]:
# import cv2
# import pandas as pd
# import numpy as np
# from sklearn.utils import shuffle
# import matplotlib.pyplot as plt
# %matplotlib inline

# def extract_sign(img, bbox, output_size=(32, 32)):
#     xc, yc, w, h = bbox.x, bbox.y, bbox.w, bbox.h
#     x1   = int(xc - w/2)
#     y1   = int(yc - h/2)
#     x2   = int(xc + w/2)
#     y2   = int(yc + h/2)
#     roi = cv2.resize(img[y1:y2, x1:x2], output_size)
#     return roi

# x_train, y_train = shuffle(x_train, y_train)
# fig = plt.figure(figsize=(17, 8))
# for i, label in enumerate(labels):
#     ax           = fig.add_subplot(4, 8, 1 + i, xticks=[], yticks=[])
#     idx          = np.where(y_train[:, 1] == label)[0][0]
#     img          = cv2.cvtColor(cv2.imread(x_train[idx]), cv2.COLOR_BGR2RGB)
#     box          = y_train[idx][0]
#     sign_only    = extract_sign(img, box, (32, 32)) #  just extract the sign
#     ax.set_title(label)
#     plt.imshow(sign_only)
# plt.show()